# 6. Anomaly Detection

[index](../Index.ipynb) | [prev](./05.Forecasting.ipynb) | [next](./07.Conclusions.ipynb)

**Motivation**:

While initially the idea for this system was around object detection, and then forecasting, these two concepts have organically led me to the ultimate idea: <span style="background-color: yellow;">Can object detections be used to detect anomalies and trigger useful alerts to the users?</span>

This Chapter is an analysis of two methods, which I have identified as the most useful at the moment:
- based on unusually high count of objects in a given hour
- based on the content of the frames, which contain predicted objects

## 6.1. Anomalies estimated from event counts

Flagging object detections as anomalies can be determined by unusually high number of events in a given hour for an object class (like Person, Car, Cat, etc.).

This kind of anomaly detection routine could be run in real time when objects of a specific class are detected in the video stream. System compares a number of already registered objects in an hour (for example $3$) versus a threshold (for example $9$) to determine if the count should be classified as anomalous. If it is anomalous, then system could trigger an alert to home owner.

These min and max thresholds could be also displayed in the forecast as two bars: one on left and one on the right hand side of the counts.

This task forms a univariate outlier detection problem, where system learns the thresholds from the historical counts.

This section is focused around three individual ways of solving this problem:
- IQR
- Z-Score
- Probabilistic method

Just before diving into the details of each method, it is important to understand the distribution of counts. Again, for readability, all work in this section is based on the *Person* object, but the same method can be applied to any object class.

<p style="text-align: center; margin-bottom: 0;">Fig. 6.1. Count distribution - all hours</p>
<img src="../Resources/img/person-count-dist.png" style="width: 50%;"/>

Overall this data is heavily skewed towards 0's, but this is expected. During the night or when it's dark, the number of objects is $0$ as the camera is not night-vision. In other time intervals there is just not much activity happening.

The mean $\mu$ of this population is $1.16$ and standard deviation $\sigma$ is $1.95$. Taking a square root of $\sigma$ gives $1.08$, which is close to $\mu$ and it is one of the main characteristics of the *Poisson distribution*.

Next, looking at the distribution of $\mu$'s for each hour shows a more detailed picture, where bars represent mean averages and red dots are a square root of $\sigma$.

<p style="text-align: center; margin-bottom: 0;">Fig. 6.2. Count distribution by hour</p>
<img src="../Resources/img/count-dist-by-hour.png" style="width: 60%;"/>

This picture shows quite a large spread of means by hour, and it is convincing enough to focus on anomaly detection for individual hours separately.

And looking at the frequency of counts for 4PM shows quite heavy skewness towards the left side:

<p style="text-align: center; margin-bottom: 0;">Fig. 6.3. Count distribution at 4PM</p>
<img src="../Resources/img/person-count-dist-singlehour.png" style="width: 40%;"/>

### 6.1.1. IQR

There are many statistical tools to deal with this kind of challenge. Arguably the most popular and easy to understand is IQR (Interquartile Range).

This method is used in a very popular plotting technique for outlier identification, the boxplot (Tukey 1977). This method is simple to explain, well understood and often produces satisfactory results.

First, one would calculate an *Interquartile Range (IQR)* using the difference between the third and first quartile, where first quartile ($Q1$) and third quartile ($Q3$) are the medians of lower and upper halves of the data respectively:

$$IQR=Q3-Q1$$

Then the lower and upper bounds are calculated by removing and adding $IQR*1.5$ from $Q1$ and $Q3$ respectively:

$$lowerBound=Q1-(IQR*1.5)$$

$$upperBound=Q3+(IQR*1.5)$$

And finally, values below lower or above upper bound are classified is outliers:

$$
\begin{equation}
  f(x)=\begin{cases}
    anomaly, & \text{if $x<lowerBound$ or $x>upperBound$}\\
    not-anomaly, & \text{otherwise}
  \end{cases}
\end{equation}
$$

, where $x$ is an individual count of objects in a single hour.

Below is a boxplot for the count dataset by hour:

<p style="text-align: center; margin-bottom: 0;">Fig. 6.4. IQR analysis - boxplot</p>
<img src="../Resources/img/boxplot-by-hour.png" style="width: 70%;"/>

Looking at the graph, it is clearly flagging a lot of points. After calculating the percentage of points above and below the bounds, IQR method classifies $5\%$ of observations as anomalous.

The high percentage of anomalies is related to the fact that counts for each hour are heavily skewed, and as mentioned in the *Adjusted boxplot* work by Hubert at al., 2008, boxplots suffer from False Positives when heavy skewness exists.

### 6.1.2. Adjusted boxplot for skewed distributions

In the 2008's [paper](https://wis.kuleuven.be/stat/robust/papers/2008/adjboxplot-revision.pdf) by M. Hubert et al., an alternative method has been proposed to IQR: *An Adjusted Boxplot for Skewed
Distributions*.

The procedure is quite similar to IQR, but introduces some additional steps:
- calculation of data skewness (called medcouple - MC):

$$MC = \frac{(Q3 − Q2) − (Q2 − Q1)}{Q3 − Q1}$$

For the count dataset and Person object class, this measure is $0.33$.

Then, the lower and upper bounds are calculated as follows:

$$lowerBound=Q1-h_l(MC)IQR$$
$$upperBound=Q3+h_u(MC)IQR$$

, where:

$$h_l(MC)=1.5e^{aMC}$$
$$h_u(MC)=1.5e^{bMC}$$

The authors of the paper have optimised the values for the constants $a$ and $b$ as $-4$ and $3$, in a way that fences mark 0.7% observations as outliers.

Applying these calculations to the count dataset classifies $148$ observations as outliers, which represents $3.5\%$ of the dataset and still generates too many False Positives.

### 6.1.2. Z-Score

Z-Score determines how many standard deviations $\sigma%$ are points $X$ away from the mean $\mu$.

$$
zScore_i=(x_i-\mu)\div\sigma
$$

, where $x_i$ is an i-th data point, $\mu$ and $\sigma$ are a sample arithmetic mean and standard deviation respectively.

Z-Score has quite interesting properties when applied to Normal distributions, where $99.7\%$ of the data points lie between +/- 3 $\sigma$'s.

In case if the skewed count dataset it also performs quite well and identifies $75$ outliers, which represents $2\%$ of the dataset.

### 6.1.3. Probabilistic method

Probabilistic models utilise *Bayesian Theorem* to derive the following formula from the Conditional Probability theory:

$$P(A|B)=\frac{P(B|A)P(A)}{P(B)}$$

Where:
- $P(A|B)$ is the posterior, meaning conditional probability of event $A$ given that $B$ is true
- $P(B|A)$ is the likelihood, also conditional probability of event $B$ ocurring given $A$ is true
- $P(A)$ is the prior (information we already know about the data)
- $P(B)$ is the marginal probability of observing event $B$

There are many benefits from using probabilistic modelling. Some of them are included below:
- no assumptions made about the distribution of the data
- it allows us to provide prior information to the model about distributions
- it does not require a lot of data
- it gives us the predictions and the uncertainty about them

**Prior**

In probabilistic programming we use the prior information we already have (like the distribution of the outcome random variable), then we define the likelihood, which tells the library how to sample the probability space given the data, and then we perform an analysis of the posterior, which contains N-samples drawn from the distibution.

In relation to the count dataset, I have identified a $2$ candidate distrubutions, which can be used as a prior in the model:

- Half Student T distribution with parameters $\sigma=1.0$ and $\nu=1.0$ and density function:

$$f(t)=\frac{\gamma(\frac{\nu + 1}{2})}{\sqrt{\nu \pi} \Gamma (\frac{\nu}{2})} (1 + \frac{t^2}{\nu})^{-\frac{\nu + 1}{2}}$$

, where $\nu$ is the number of degrees of freedom and $\Gamma$ is the gamma function.

- Gamma distribution with parameters $\alpha=1.5$ (shape) and $\beta=0.5$ (rate) and density function:

$$f(x;\alpha;\beta)=\frac{\beta^\alpha x^{\alpha-1} e^{-\beta x}}{\Gamma(\alpha)}$$

, where $x>0$, $\alpha,\beta > 0$ and $\Gamma(\alpha)$ is the gamma function

Below is the multi-plot with both distrubutions and the true dataset with counts between 1PM and 3PM:

<p style="text-align: center; margin-bottom: 5px;">Fig. 6.5. Priors selection</p>
<img src="../Resources/img/anomaly-det-priors.png" style="width: 70%;"/>

Based on the graph above, Gamma distibution with $\alpha=1.8$ and $\beta=0.8$ seems to be more suitable to this distribution.

**Likelihood**

The next item we need is the likelihood function, which is used to estimate the counts for every hour, given the data $X$.

A suitable likelihood function will use the Poisson process.

Poisson is a discrete probability distribution, which is used when we need to model a number of event occuring in a time interval.

As per [Wiki page about Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution), probability mass function of $X$ for $k=0,1,2,3, ...$ is given by:

$$f(k;\lambda) = Pr(X=k) = \frac{\lambda^{k} e^{-\lambda}}{k!})$$

, where $\lambda>0$, *expected value* and *variance* are both equal to $\lambda$, e is Euler's number ($e=2.718...$) and $k!$ is the factorial of k.

The likelihood function for Poisson process is given by:

$$L(\lambda;x_1,...,x_n)=\prod^{n}_{j=1}exp(-\lambda)\frac{1}{x_j!}\lambda^{x_j}$$

As highlighted in the [online.stat.psu.edu article](https://online.stat.psu.edu/stat504/node/27/), likelihood is a tool for summarizing the data’s evidence about unknown parameters, and often (due to computational convenience), it is transformed into log-likelihood.

The log-likelihood for the Poisson process is given by:

$$l(\lambda;x_1,...,x_n)=-n \lambda - \sum^n_{j=1}ln(x_j!)+ln(\lambda)\sum^n_{j=1}x_j$$

Now coding up the solution is made very simple with libraries for Probabilistic Programming, like `PyMC3`:
- first define a Gamma prior (it is possible to have a list of $24$ priors - one for each hour)
- then define a list Poisson likelihood functions (again, $1$ for each hour)
- finally, sample from the posterior and analyse results

Before going into the results, I would like to explain why sampling is used by PyMC3 and which sampling method is appropriate to which kind of data.

In order to compute the optimized values for the model's parameters (also called maximum a posteriori, or MAP), there are two paths to take:
- numerical optimization methods, which are usually fast and easy (`find_map` function in PyMC3)

Default optimization algorithm is BFGS (Broyden–Fletcher–Goldfarb–Shanno, Fletcher, Roger, 1987), but other functions from `scipy.optimize` are acceptable. The downside of this approach is that it often finds only local optima, and as advised in [PyMC3 documentation](https://docs.pymc.io/notebooks/getting_started.html), this method only exists for historical reasons. The second limitation is a lack of uncertainty measure, as only a single value for each parameter is returned.

- sampling based optimization used for more complex scenarios (`sample` function in PyMC3)

This method is a recommended, simulation-based approach with a few algorithms suitable for different problems:
- Binary variables will be assigned to BinaryMetropolis
- Discrete variables will be assigned to Metropolis
- Continuous variables will be assigned to NUTS (No-U-Turn Sampler)

The `sample` function return a `trace` object, which can be queried to obtain the samples for individual parameter. A standard deviation of these values can be interpreted as an uncertainty.

Sampling process for the count data dataset takes less than $30$ seconds.

Below are some of the most useful statistics (like mean, standard deviation, median and quantiles) for each hour, which can be easily generated by PyMC3 with the use of `pm.summary` method:

<p style="text-align: center; margin-bottom: 5px;">Fig. 6.6. PyMC3 - Posterior stats</p>
<img src="../Resources/img/posterior_stats.png" style="width: 90%;"/>

Next, one can take an advantage of having multiple samples for the estimated rate $\lambda$, and generate $N$ counts for all these rates (using all sampled rates for a single hour embeds the uncertainty into the generated counts).

Probability density for the 4PM then can be plotted and questions asked about the probability of obtaining a count $K$:

<p style="text-align: center; margin-bottom: 5px;">Fig. 6.7. Probability density for 4PM</p>
<img src="../Resources/img/4pm-prob-dens.png" style="width: 60%;"/>

For example, if $K=8$, then there is $1\%$ chance to see an $8$ or more objects at 4PM.

Now, the final idea here is to define a percentage of observations, which need to be classified as anomalies, and use the approach from above, but this time apply it to all hours.

Once this is set to $0.001$, $61$ anomalous observations are detected, which represents $1.5\%$ of the dataset. The result is the max count fence for each hour, above which counts become anomalies. For example the fence for 4PM has been determined as $9.0$.

To visualise the results for all hours I have generated a plot, with a dashed line representing the threshold for anomalies. Red dots are anomalies, and their size corresponds to their magnitude:

<p style="text-align: center; margin-bottom: 0;">Fig. 6.8. Anomaly thresholds by hour</p>
<img src="../Resources/img/anomaly-thresholds.png" style="width: 80%;"/>

### 6.1.4. Summary

It is very clear that the Probabilistic Model is the most flexible and gives the most interesting opportunities based on the samples, which it has generated.

The percentage of anomalies is also fully under control, and can be made as strict, or as relaxed as one would like.

The next step in this section would be to actually emded the max count fences in the hourly forecast graph, so it is clear for the users of the system when to expect an alert for each hour, and why an alert has been triggered.

## 6.2. Anomalies estimated from frames content

[index](../Index.ipynb) | [prev](./05.Forecasting.ipynb) | [next](./07.Conclusions.ipynb)